<a href="https://colab.research.google.com/github/winterForestStump/thesis/blob/main/retrieval/Retrievals_approaches_evaluation_phi3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 MB 14.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.6 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.75-cp310-cp310-linux_x86_64.whl size=76330307 sha256=634454f0e770ad26a674e9fbe0ebf715ed56ade37dbb083b8f64dd1b88288821
  Stored in directory: /root/.cache/pip/wheels/5e/df/9a/e4bb2e48bfa64fb174f0f786296c8507dbebea2a112f1adf8d
Successfully built llama-cpp-python


In [2]:
!huggingface-cli download microsoft/Phi-3-mini-4k-instruct-gguf Phi-3-mini-4k-instruct-fp16.gguf --local-dir ./models --local-dir-use-symlinks False

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-fp16.gguf to /root/.cache/huggingface/hub/tmpfyp_gdsx
Phi-3-mini-4k-instruct-fp16.gguf: 100% 7.64G/7.64G [01:22<00:00, 93.0MB/s]
./models/Phi-3-mini-4k-instruct-fp16.gguf


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!pip install langchain langchain-core langchain-community --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.7/973.7 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.9/307.9 kB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 18.2 MB/s eta 0:00:00


In [5]:
from langchain_community.llms import LlamaCpp
from langchain.prompts import PromptTemplate
from langchain_community.llms import LlamaCpp
from langchain_core.output_parsers import JsonOutputParser
from json import JSONDecodeError
from langchain_core.exceptions import OutputParserException
import pandas as pd
import os

In [6]:
TEMP = 0
N_CTX = 4096
N_GPU_L = -1

llm = LlamaCpp(
    model_path="/content/models/Phi-3-mini-4k-instruct-fp16.gguf",
    temperature=TEMP,
    n_ctx=N_CTX,
    n_gpu_layers = N_GPU_L,
    verbose=True
)

llama_model_loader: loaded meta data with 23 key-value pairs and 195 tensors from /content/models/Phi-3-mini-4k-instruct-fp16.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = phi3
llama_model_loader: - kv   1:                               general.name str              = Phi3
llama_model_loader: - kv   2:                        phi3.context_length u32              = 4096
llama_model_loader: - kv   3:                      phi3.embedding_length u32              = 3072
llama_model_loader: - kv   4:                   phi3.feed_forward_length u32              = 8192
llama_model_loader: - kv   5:                           phi3.block_count u32              = 32
llama_model_loader: - kv   6:                  phi3.attention.head_count u32              = 32
llama_model_loader: - kv   7:               phi3.attention.head_count

In [7]:
### Retrieval Grader for approach 1

prompt_1 = PromptTemplate(
    template="""
    <|assistant|> You are a grader assessing relevance of a retrieved document to a user question. If the document contains keywords related to the user question, grade it as relevant.
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explaination. <|end|>
    <|user|> Here is the retrieved document: \n\n {document} \n\n Here is the user question: {question} <|end|>
    <|assistant|>
    """,
    input_variables=["question", "document"],
)

retrieval_grader_1 = prompt_1 | llm | JsonOutputParser()

In [8]:
### Retrieval Grader for approach 2

prompt_2 = PromptTemplate(
    template="""
    <|assistant|> You are a grader assessing relevance of a retrieved document to a user question. If the document contains keywords related to the user question, grade it as relevant.
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explaination. <|end|>
    <|user|> Here is the retrieved document: \n\n {document} \n\n Here is the user question: {question_name} <|end|>
    <|assistant|>
    """,
    input_variables=["question_name", "document"],
)

retrieval_grader_2 = prompt_2 | llm | JsonOutputParser()

In [9]:
folder_path = '/content/drive/MyDrive/Thesis/retrievers/'

dataframes = []
for file_name in os.listdir(folder_path):
    if file_name.endswith('.json'):
        approach_name = file_name
        dataframes.append(approach_name)

In [10]:
def calculate_results(approach_number:int, distance: str):
  results_list = []

  for i in range(len(dataframes)):

    if approach_number == 1:
      approach = pd.read_json(os.path.join(folder_path + dataframes[i]))
      try:
        for j in range(len(approach)):
          question = approach['question'][j]
          doc_txt = approach[distance][j]['page_content']
          try:
            answer = retrieval_grader_1.invoke({"question": question, "document": doc_txt})
            results_list.append(pd.DataFrame({"approach": 1,"distance": distance, "question": question, "answer": [answer], "document": doc_txt}))
          except JSONDecodeError as e:
            print(f"JSONDecodeError occurred for question: {question}. Skipping...")
            continue
          except OutputParserException as e:
            print(f"OutputParserException occurred for question: {question}. Skipping...")
            continue
      except KeyError as e:
        print(f"KeyError occurred: {e}. Skipping approach {approach_number} for dataframe {dataframes[i]}")
        continue


    elif approach_number == 2:
      approach = pd.read_json(os.path.join(folder_path + dataframes[i]))
      try:
        for j in range(len(approach)):
          question = approach['question_name'][j]
          doc_txt = approach[distance][j]['page_content']
          try:
            answer = retrieval_grader_2.invoke({"question_name": question, "document": doc_txt})
            results_list.append(pd.DataFrame({"approach": 2,"distance": distance, "question": question, "answer": [answer], "document": doc_txt}))
          except JSONDecodeError as e:
            print(f"JSONDecodeError occurred for question: {question}. Skipping...")
            continue
          except OutputParserException as e:
            print(f"OutputParserException occurred for question: {question}. Skipping...")
            continue
      except KeyError as e:
        print(f"KeyError occurred: {e}. Skipping approach {approach_number} for dataframe {dataframes[i]}")
        continue

  if results_list:
    results = pd.concat(results_list, ignore_index=True)
    score = 0
    for i in range(len(results)):
      if (results['answer'][i] is not None) and (results['answer'][i]['score'] == 'yes'):
        score += 1
    total_score = score/len(results)
    print(f'Total score for approach {approach_number} and distance function {distance} is {total_score}')
  else:
    print(f"No results to concatenate for approach {approach_number} and distance {distance}")

In [11]:
calculate_results(1, 'cosine')


llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       6.78 ms /     7 runs   (    0.97 ms per token,  1032.14 tokens per second)
llama_print_timings: prompt eval time =    4510.92 ms /   792 tokens (    5.70 ms per token,   175.57 tokens per second)
llama_print_timings:        eval time =     298.50 ms /     6 runs   (   49.75 ms per token,    20.10 tokens per second)
llama_print_timings:       total time =    5298.46 ms /   798 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       8.28 ms /     7 runs   (    1.18 ms per token,   845.82 tokens per second)
llama_print_timings: prompt eval time =    3427.63 ms /   613 tokens (    5.59 ms per token,   178.84 tokens per second)
llama_print_timings:        eval time =     236.69 ms /     6 runs   (   39.45 ms per token,    25.35 tokens per second)
llama_print_timings:       total time =    4125.97 ms /   619 

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_cosine_2.json
KeyError occurred: 'cosine'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_ip_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_ip_2.json
KeyError occurred: 'cosine'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_l2_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_l2_2.json


Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       4.53 ms /     7 runs   (    0.65 ms per token,  1543.89 tokens per second)
llama_print_timings: prompt eval time =    2754.76 ms /   520 tokens (    5.30 ms per token,   188.76 tokens per second)
llama_print_timings:        eval time =     233.41 ms /     6 runs   (   38.90 ms per token,    25.71 tokens per second)
llama_print_timings:       total time =    3273.27 ms /   526 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       4.03 ms /     7 runs   (    0.58 ms per token,  1738.70 tokens per second)
llama_print_timings: prompt eval time =    3491.75 ms /   659 tokens (    5.30 ms per token,   188.73 tokens per second)
llama_print_timings:        eval time =     232.64 ms /     6 runs   (   38.77 ms per token,    25.79 tokens per second)
llama_print_timings:       to

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_cosine_2.json
KeyError occurred: 'cosine'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_ip_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_ip_2.json
KeyError occurred: 'cosine'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_l2_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_l2_2.json


Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.78 ms /     7 runs   (    0.54 ms per token,  1849.89 tokens per second)
llama_print_timings: prompt eval time =    2929.04 ms /   556 tokens (    5.27 ms per token,   189.82 tokens per second)
llama_print_timings:        eval time =     232.89 ms /     6 runs   (   38.82 ms per token,    25.76 tokens per second)
llama_print_timings:       total time =    3345.51 ms /   562 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.87 ms /     7 runs   (    0.55 ms per token,  1810.19 tokens per second)
llama_print_timings: prompt eval time =    2894.03 ms /   551 tokens (    5.25 ms per token,   190.39 tokens per second)
llama_print_timings:        eval time =     233.79 ms /     6 runs   (   38.97 ms per token,    25.66 tokens per second)
llama_print_timings:       to

...

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_GENERAL MILLS INC_cosine_2.json
KeyError occurred: 'cosine'. Skipping approach 1 for dataframe results_approach_GENERAL MILLS INC_ip_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_GENERAL MILLS INC_ip_2.json
KeyError occurred: 'cosine'. Skipping approach 1 for dataframe results_approach_GENERAL MILLS INC_l2_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_GENERAL MILLS INC_l2_2.json


Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.02 ms /     6 runs   (    0.50 ms per token,  1984.78 tokens per second)
llama_print_timings: prompt eval time =    4181.85 ms /   776 tokens (    5.39 ms per token,   185.56 tokens per second)
llama_print_timings:        eval time =     237.92 ms /     6 runs   (   39.65 ms per token,    25.22 tokens per second)
llama_print_timings:       total time =    4808.51 ms /   782 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.31 ms /     6 runs   (    0.55 ms per token,  1812.14 tokens per second)
llama_print_timings: prompt eval time =    3304.42 ms /   605 tokens (    5.46 ms per token,   183.09 tokens per second)
llama_print_timings:        eval time =     198.93 ms /     5 runs   (   39.79 ms per token,    25.13 tokens per second)
llama_print_timings:       to

OutputParserException occurred for question: What is the company's accounts receivable turnover and are there any concerns regarding receivables aging?. Skipping...



llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       4.01 ms /     7 runs   (    0.57 ms per token,  1744.33 tokens per second)
llama_print_timings: prompt eval time =    2329.69 ms /   445 tokens (    5.24 ms per token,   191.01 tokens per second)
llama_print_timings:        eval time =     230.10 ms /     6 runs   (   38.35 ms per token,    26.08 tokens per second)
llama_print_timings:       total time =    2717.07 ms /   451 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.88 ms /     7 runs   (    0.55 ms per token,  1804.12 tokens per second)
llama_print_timings: prompt eval time =    2329.21 ms /   446 tokens (    5.22 ms per token,   191.48 tokens per second)
llama_print_timings:        eval time =     233.88 ms /     6 runs   (   38.98 ms per token,    25.65 tokens per second)
llama_print_timings:       total time =    2726.42 ms /   452 

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_Walmart Inc._cosine_2.json
KeyError occurred: 'cosine'. Skipping approach 1 for dataframe results_approach_Walmart Inc._ip_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_Walmart Inc._ip_2.json
KeyError occurred: 'cosine'. Skipping approach 1 for dataframe results_approach_Walmart Inc._l2_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_Walmart Inc._l2_2.json
Total score for approach 1 and distance function cosine is 0.6194968553459119


In [12]:
calculate_results(1, 'ip')

KeyError occurred: 'ip'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_cosine_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_cosine_2.json


Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       4.12 ms /     7 runs   (    0.59 ms per token,  1701.09 tokens per second)
llama_print_timings: prompt eval time =    3557.86 ms /   659 tokens (    5.40 ms per token,   185.22 tokens per second)
llama_print_timings:        eval time =     235.15 ms /     6 runs   (   39.19 ms per token,    25.52 tokens per second)
llama_print_timings:       total time =    4049.39 ms /   665 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.99 ms /     7 runs   (    0.57 ms per token,  1755.71 tokens per second)
llama_print_timings: prompt eval time =    3269.19 ms /   613 tokens (    5.33 ms per token,   187.51 tokens per second)
llama_print_timings:        eval time =     232.45 ms /     6 runs   (   38.74 ms per token,    25.81 tokens per second)
llama_print_timings:       to

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_ip_2.json
KeyError occurred: 'ip'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_l2_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_l2_2.json
KeyError occurred: 'ip'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_cosine_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_cosine_2.json



llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.70 ms /     7 runs   (    0.53 ms per token,  1891.89 tokens per second)
llama_print_timings: prompt eval time =    2715.25 ms /   520 tokens (    5.22 ms per token,   191.51 tokens per second)
llama_print_timings:        eval time =     234.82 ms /     6 runs   (   39.14 ms per token,    25.55 tokens per second)
llama_print_timings:       total time =    3121.66 ms /   526 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       4.53 ms /     7 runs   (    0.65 ms per token,  1544.23 tokens per second)
llama_print_timings: prompt eval time =    3524.21 ms /   659 tokens (    5.35 ms per token,   186.99 tokens per second)
llama_print_timings:        eval time =     236.65 ms /     6 runs   (   39.44 ms per token,    25.35 tokens per second)
llama_print_timings:       total time =    4081.92 ms /   665 

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_ip_2.json
KeyError occurred: 'ip'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_l2_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_l2_2.json
KeyError occurred: 'ip'. Skipping approach 1 for dataframe results_approach_PayPal Holdings, Inc._cosine_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_PayPal Holdings, Inc._cosine_2.json


...

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_GENERAL MILLS INC_ip_2.json
KeyError occurred: 'ip'. Skipping approach 1 for dataframe results_approach_GENERAL MILLS INC_l2_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_GENERAL MILLS INC_l2_2.json
KeyError occurred: 'ip'. Skipping approach 1 for dataframe results_approach_Walmart Inc._cosine_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_Walmart Inc._cosine_2.json



llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.03 ms /     6 runs   (    0.51 ms per token,  1979.54 tokens per second)
llama_print_timings: prompt eval time =    4166.65 ms /   776 tokens (    5.37 ms per token,   186.24 tokens per second)
llama_print_timings:        eval time =     237.57 ms /     6 runs   (   39.59 ms per token,    25.26 tokens per second)
llama_print_timings:       total time =    4633.87 ms /   782 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       4.41 ms /     6 runs   (    0.73 ms per token,  1362.09 tokens per second)
llama_print_timings: prompt eval time =    3317.12 ms /   605 tokens (    5.48 ms per token,   182.39 tokens per second)
llama_print_timings:        eval time =     198.74 ms /     5 runs   (   39.75 ms per token,    25.16 tokens per second)
llama_print_timings:       total time =    3777.24 ms /   610 

OutputParserException occurred for question: What is the company's accounts receivable turnover and are there any concerns regarding receivables aging?. Skipping...



llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       4.56 ms /     7 runs   (    0.65 ms per token,  1536.44 tokens per second)
llama_print_timings: prompt eval time =    2346.25 ms /   445 tokens (    5.27 ms per token,   189.66 tokens per second)
llama_print_timings:        eval time =     233.14 ms /     6 runs   (   38.86 ms per token,    25.74 tokens per second)
llama_print_timings:       total time =    2822.77 ms /   451 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       4.17 ms /     7 runs   (    0.60 ms per token,  1678.66 tokens per second)
llama_print_timings: prompt eval time =    2336.81 ms /   446 tokens (    5.24 ms per token,   190.86 tokens per second)
llama_print_timings:        eval time =     232.51 ms /     6 runs   (   38.75 ms per token,    25.80 tokens per second)
llama_print_timings:       total time =    2721.90 ms /   452 

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_Walmart Inc._ip_2.json
KeyError occurred: 'ip'. Skipping approach 1 for dataframe results_approach_Walmart Inc._l2_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_Walmart Inc._l2_2.json
Total score for approach 1 and distance function ip is 0.6194968553459119


In [13]:
calculate_results(1, 'l2')

Llama.generate: prefix-match hit


KeyError occurred: 'l2'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_cosine_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_cosine_2.json
KeyError occurred: 'l2'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_ip_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_ip_2.json



llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.43 ms /     7 runs   (    0.49 ms per token,  2042.60 tokens per second)
llama_print_timings: prompt eval time =    3548.58 ms /   659 tokens (    5.38 ms per token,   185.71 tokens per second)
llama_print_timings:        eval time =     235.74 ms /     6 runs   (   39.29 ms per token,    25.45 tokens per second)
llama_print_timings:       total time =    4100.69 ms /   665 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.81 ms /     7 runs   (    0.54 ms per token,  1836.79 tokens per second)
llama_print_timings: prompt eval time =    3258.13 ms /   613 tokens (    5.32 ms per token,   188.14 tokens per second)
llama_print_timings:        eval time =     235.61 ms /     6 runs   (   39.27 ms per token,    25.47 tokens per second)
llama_print_timings:       total time =    3680.07 ms /   619 

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_COCA COLA CO_l2_2.json
KeyError occurred: 'l2'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_cosine_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_cosine_2.json
KeyError occurred: 'l2'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_ip_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_AMAZON COM INC_ip_2.json



llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       4.54 ms /     7 runs   (    0.65 ms per token,  1542.87 tokens per second)
llama_print_timings: prompt eval time =    2742.84 ms /   520 tokens (    5.27 ms per token,   189.58 tokens per second)
llama_print_timings:        eval time =     235.18 ms /     6 runs   (   39.20 ms per token,    25.51 tokens per second)
llama_print_timings:       total time =    3231.25 ms /   526 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       4.17 ms /     7 runs   (    0.60 ms per token,  1679.06 tokens per second)
llama_print_timings: prompt eval time =    3534.62 ms /   659 tokens (    5.36 ms per token,   186.44 tokens per second)
llama_print_timings:        eval time =     238.40 ms /     6 runs   (   39.73 ms per token,    25.17 tokens per second)
llama_print_timings:       total time =    4035.55 ms /   665 

...

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_GENERAL MILLS INC_l2_2.json
KeyError occurred: 'l2'. Skipping approach 1 for dataframe results_approach_Walmart Inc._cosine_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_Walmart Inc._cosine_2.json
KeyError occurred: 'l2'. Skipping approach 1 for dataframe results_approach_Walmart Inc._ip_1.json
KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_Walmart Inc._ip_2.json



llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.66 ms /     6 runs   (    0.61 ms per token,  1641.14 tokens per second)
llama_print_timings: prompt eval time =    4169.22 ms /   776 tokens (    5.37 ms per token,   186.13 tokens per second)
llama_print_timings:        eval time =     239.53 ms /     6 runs   (   39.92 ms per token,    25.05 tokens per second)
llama_print_timings:       total time =    4689.93 ms /   782 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.11 ms /     6 runs   (    0.52 ms per token,  1928.64 tokens per second)
llama_print_timings: prompt eval time =    3326.67 ms /   605 tokens (    5.50 ms per token,   181.86 tokens per second)
llama_print_timings:        eval time =     200.06 ms /     5 runs   (   40.01 ms per token,    24.99 tokens per second)
llama_print_timings:       total time =    3791.34 ms /   610 

OutputParserException occurred for question: What is the company's accounts receivable turnover and are there any concerns regarding receivables aging?. Skipping...



llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.94 ms /     7 runs   (    0.56 ms per token,  1774.85 tokens per second)
llama_print_timings: prompt eval time =    2335.31 ms /   445 tokens (    5.25 ms per token,   190.55 tokens per second)
llama_print_timings:        eval time =     234.84 ms /     6 runs   (   39.14 ms per token,    25.55 tokens per second)
llama_print_timings:       total time =    2720.92 ms /   451 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.91 ms /     7 runs   (    0.56 ms per token,  1792.11 tokens per second)
llama_print_timings: prompt eval time =    2336.30 ms /   446 tokens (    5.24 ms per token,   190.90 tokens per second)
llama_print_timings:        eval time =     233.97 ms /     6 runs   (   38.99 ms per token,    25.64 tokens per second)
llama_print_timings:       total time =    2714.71 ms /   452 

KeyError occurred: 'question'. Skipping approach 1 for dataframe results_approach_Walmart Inc._l2_2.json
Total score for approach 1 and distance function l2 is 0.6194968553459119


In [14]:
calculate_results(2, 'cosine')

Llama.generate: prefix-match hit


KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_COCA COLA CO_cosine_1.json



llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.01 ms /     6 runs   (    0.50 ms per token,  1992.03 tokens per second)
llama_print_timings: prompt eval time =    5996.84 ms /  1083 tokens (    5.54 ms per token,   180.60 tokens per second)
llama_print_timings:        eval time =     202.60 ms /     5 runs   (   40.52 ms per token,    24.68 tokens per second)
llama_print_timings:       total time =    6517.91 ms /  1088 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       4.41 ms /     7 runs   (    0.63 ms per token,  1588.74 tokens per second)
llama_print_timings: prompt eval time =    5049.03 ms /   926 tokens (    5.45 ms per token,   183.40 tokens per second)
llama_print_timings:        eval time =     242.53 ms /     6 runs   (   40.42 ms per token,    24.74 tokens per second)
llama_print_timings:       total time =    5655.96 ms /   932 

OutputParserException occurred for question: Are there any tax-related risks or benefits for the COCA COLA CO mentioned?. Skipping...



llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.26 ms /     6 runs   (    0.54 ms per token,  1838.24 tokens per second)
llama_print_timings: prompt eval time =    3346.88 ms /   629 tokens (    5.32 ms per token,   187.94 tokens per second)
llama_print_timings:        eval time =     198.49 ms /     5 runs   (   39.70 ms per token,    25.19 tokens per second)
llama_print_timings:       total time =    3737.03 ms /   634 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       6.13 ms /     9 runs   (    0.68 ms per token,  1468.43 tokens per second)
llama_print_timings: prompt eval time =    3939.17 ms /   735 tokens (    5.36 ms per token,   186.59 tokens per second)
llama_print_timings:        eval time =     317.57 ms /     8 runs   (   39.70 ms per token,    25.19 tokens per second)
llama_print_timings:       total time =    4510.10 ms /   743 

...

KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_GENERAL MILLS INC_ip_1.json
KeyError occurred: 'cosine'. Skipping approach 2 for dataframe results_approach_GENERAL MILLS INC_ip_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_GENERAL MILLS INC_l2_1.json
KeyError occurred: 'cosine'. Skipping approach 2 for dataframe results_approach_GENERAL MILLS INC_l2_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_Walmart Inc._cosine_1.json



llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.04 ms /     6 runs   (    0.51 ms per token,  1975.63 tokens per second)
llama_print_timings: prompt eval time =    5986.38 ms /  1084 tokens (    5.52 ms per token,   181.08 tokens per second)
llama_print_timings:        eval time =     204.40 ms /     5 runs   (   40.88 ms per token,    24.46 tokens per second)
llama_print_timings:       total time =    6548.68 ms /  1089 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.13 ms /     6 runs   (    0.52 ms per token,  1917.55 tokens per second)
llama_print_timings: prompt eval time =    5837.12 ms /  1050 tokens (    5.56 ms per token,   179.88 tokens per second)
llama_print_timings:        eval time =     203.37 ms /     5 runs   (   40.67 ms per token,    24.59 tokens per second)
llama_print_timings:       total time =    6529.72 ms /  1055 

OutputParserException occurred for question: What employee benefits does the Walmart Inc. offer?. Skipping...



llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.94 ms /     7 runs   (    0.56 ms per token,  1776.20 tokens per second)
llama_print_timings: prompt eval time =    2591.96 ms /   492 tokens (    5.27 ms per token,   189.82 tokens per second)
llama_print_timings:        eval time =     235.17 ms /     6 runs   (   39.19 ms per token,    25.51 tokens per second)
llama_print_timings:       total time =    2991.61 ms /   498 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.33 ms /     6 runs   (    0.55 ms per token,  1802.88 tokens per second)
llama_print_timings: prompt eval time =    3723.47 ms /   690 tokens (    5.40 ms per token,   185.31 tokens per second)
llama_print_timings:        eval time =     195.28 ms /     5 runs   (   39.06 ms per token,    25.60 tokens per second)
llama_print_timings:       total time =    4258.97 ms /   695 

KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_Walmart Inc._ip_1.json
KeyError occurred: 'cosine'. Skipping approach 2 for dataframe results_approach_Walmart Inc._ip_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_Walmart Inc._l2_1.json
KeyError occurred: 'cosine'. Skipping approach 2 for dataframe results_approach_Walmart Inc._l2_2.json
Total score for approach 2 and distance function cosine is 0.4055727554179567


In [15]:
calculate_results(2, 'ip')

KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_COCA COLA CO_cosine_1.json
KeyError occurred: 'ip'. Skipping approach 2 for dataframe results_approach_COCA COLA CO_cosine_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_COCA COLA CO_ip_1.json


Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       2.95 ms /     6 runs   (    0.49 ms per token,  2034.59 tokens per second)
llama_print_timings: prompt eval time =    5998.57 ms /  1083 tokens (    5.54 ms per token,   180.54 tokens per second)
llama_print_timings:        eval time =     201.68 ms /     5 runs   (   40.34 ms per token,    24.79 tokens per second)
llama_print_timings:       total time =    6582.03 ms /  1088 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       4.57 ms /     7 runs   (    0.65 ms per token,  1532.06 tokens per second)
llama_print_timings: prompt eval time =    5040.94 ms /   926 tokens (    5.44 ms per token,   183.70 tokens per second)
llama_print_timings:        eval time =     242.85 ms /     6 runs   (   40.48 ms per token,    24.71 tokens per second)
llama_print_timings:       to

OutputParserException occurred for question: Are there any tax-related risks or benefits for the COCA COLA CO mentioned?. Skipping...



llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.47 ms /     6 runs   (    0.58 ms per token,  1727.12 tokens per second)
llama_print_timings: prompt eval time =    3345.74 ms /   629 tokens (    5.32 ms per token,   188.00 tokens per second)
llama_print_timings:        eval time =     197.29 ms /     5 runs   (   39.46 ms per token,    25.34 tokens per second)
llama_print_timings:       total time =    3738.31 ms /   634 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       4.65 ms /     9 runs   (    0.52 ms per token,  1935.07 tokens per second)
llama_print_timings: prompt eval time =    3930.65 ms /   735 tokens (    5.35 ms per token,   186.99 tokens per second)
llama_print_timings:        eval time =     318.20 ms /     8 runs   (   39.78 ms per token,    25.14 tokens per second)
llama_print_timings:       total time =    4471.87 ms /   743 

KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_COCA COLA CO_l2_1.json
KeyError occurred: 'ip'. Skipping approach 2 for dataframe results_approach_COCA COLA CO_l2_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_AMAZON COM INC_cosine_1.json
KeyError occurred: 'ip'. Skipping approach 2 for dataframe results_approach_AMAZON COM INC_cosine_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_AMAZON COM INC_ip_1.json


...

KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_GENERAL MILLS INC_l2_1.json
KeyError occurred: 'ip'. Skipping approach 2 for dataframe results_approach_GENERAL MILLS INC_l2_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_Walmart Inc._cosine_1.json
KeyError occurred: 'ip'. Skipping approach 2 for dataframe results_approach_Walmart Inc._cosine_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_Walmart Inc._ip_1.json



llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.01 ms /     6 runs   (    0.50 ms per token,  1990.71 tokens per second)
llama_print_timings: prompt eval time =    6002.37 ms /  1084 tokens (    5.54 ms per token,   180.60 tokens per second)
llama_print_timings:        eval time =     203.37 ms /     5 runs   (   40.67 ms per token,    24.59 tokens per second)
llama_print_timings:       total time =    6693.41 ms /  1089 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.19 ms /     6 runs   (    0.53 ms per token,  1879.70 tokens per second)
llama_print_timings: prompt eval time =    5829.96 ms /  1050 tokens (    5.55 ms per token,   180.10 tokens per second)
llama_print_timings:        eval time =     203.74 ms /     5 runs   (   40.75 ms per token,    24.54 tokens per second)
llama_print_timings:       total time =    6382.70 ms /  1055 

OutputParserException occurred for question: What employee benefits does the Walmart Inc. offer?. Skipping...



llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.85 ms /     7 runs   (    0.55 ms per token,  1820.55 tokens per second)
llama_print_timings: prompt eval time =    2607.08 ms /   492 tokens (    5.30 ms per token,   188.72 tokens per second)
llama_print_timings:        eval time =     232.78 ms /     6 runs   (   38.80 ms per token,    25.78 tokens per second)
llama_print_timings:       total time =    3092.01 ms /   498 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.23 ms /     6 runs   (    0.54 ms per token,  1857.59 tokens per second)
llama_print_timings: prompt eval time =    3702.62 ms /   690 tokens (    5.37 ms per token,   186.35 tokens per second)
llama_print_timings:        eval time =     195.94 ms /     5 runs   (   39.19 ms per token,    25.52 tokens per second)
llama_print_timings:       total time =    4117.58 ms /   695 

KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_Walmart Inc._l2_1.json
KeyError occurred: 'ip'. Skipping approach 2 for dataframe results_approach_Walmart Inc._l2_2.json
Total score for approach 2 and distance function ip is 0.4055727554179567


In [16]:
calculate_results(2, 'l2')

Llama.generate: prefix-match hit


KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_COCA COLA CO_cosine_1.json
KeyError occurred: 'l2'. Skipping approach 2 for dataframe results_approach_COCA COLA CO_cosine_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_COCA COLA CO_ip_1.json
KeyError occurred: 'l2'. Skipping approach 2 for dataframe results_approach_COCA COLA CO_ip_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_COCA COLA CO_l2_1.json



llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.17 ms /     6 runs   (    0.53 ms per token,  1892.15 tokens per second)
llama_print_timings: prompt eval time =    5989.88 ms /  1083 tokens (    5.53 ms per token,   180.81 tokens per second)
llama_print_timings:        eval time =     202.83 ms /     5 runs   (   40.57 ms per token,    24.65 tokens per second)
llama_print_timings:       total time =    6509.29 ms /  1088 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.72 ms /     7 runs   (    0.53 ms per token,  1883.75 tokens per second)
llama_print_timings: prompt eval time =    5054.39 ms /   926 tokens (    5.46 ms per token,   183.21 tokens per second)
llama_print_timings:        eval time =     241.87 ms /     6 runs   (   40.31 ms per token,    24.81 tokens per second)
llama_print_timings:       total time =    5690.79 ms /   932 

OutputParserException occurred for question: Are there any tax-related risks or benefits for the COCA COLA CO mentioned?. Skipping...



llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.29 ms /     6 runs   (    0.55 ms per token,  1825.93 tokens per second)
llama_print_timings: prompt eval time =    3350.15 ms /   629 tokens (    5.33 ms per token,   187.75 tokens per second)
llama_print_timings:        eval time =     196.61 ms /     5 runs   (   39.32 ms per token,    25.43 tokens per second)
llama_print_timings:       total time =    3737.50 ms /   634 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       4.90 ms /     9 runs   (    0.54 ms per token,  1837.11 tokens per second)
llama_print_timings: prompt eval time =    3955.57 ms /   735 tokens (    5.38 ms per token,   185.81 tokens per second)
llama_print_timings:        eval time =     317.85 ms /     8 runs   (   39.73 ms per token,    25.17 tokens per second)
llama_print_timings:       total time =    4630.33 ms /   743 

...

KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_Walmart Inc._cosine_1.json
KeyError occurred: 'l2'. Skipping approach 2 for dataframe results_approach_Walmart Inc._cosine_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_Walmart Inc._ip_1.json
KeyError occurred: 'l2'. Skipping approach 2 for dataframe results_approach_Walmart Inc._ip_2.json
KeyError occurred: 'question_name'. Skipping approach 2 for dataframe results_approach_Walmart Inc._l2_1.json



llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.35 ms /     6 runs   (    0.56 ms per token,  1788.38 tokens per second)
llama_print_timings: prompt eval time =    6015.09 ms /  1084 tokens (    5.55 ms per token,   180.21 tokens per second)
llama_print_timings:        eval time =     203.93 ms /     5 runs   (   40.79 ms per token,    24.52 tokens per second)
llama_print_timings:       total time =    6668.04 ms /  1089 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.04 ms /     6 runs   (    0.51 ms per token,  1976.28 tokens per second)
llama_print_timings: prompt eval time =    5829.43 ms /  1050 tokens (    5.55 ms per token,   180.12 tokens per second)
llama_print_timings:        eval time =     204.51 ms /     5 runs   (   40.90 ms per token,    24.45 tokens per second)
llama_print_timings:       total time =    6343.99 ms /  1055 

OutputParserException occurred for question: What employee benefits does the Walmart Inc. offer?. Skipping...



llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       4.61 ms /     7 runs   (    0.66 ms per token,  1518.77 tokens per second)
llama_print_timings: prompt eval time =    2604.42 ms /   492 tokens (    5.29 ms per token,   188.91 tokens per second)
llama_print_timings:        eval time =     234.61 ms /     6 runs   (   39.10 ms per token,    25.57 tokens per second)
llama_print_timings:       total time =    3108.06 ms /   498 tokens
Llama.generate: prefix-match hit

llama_print_timings:        load time =     450.10 ms
llama_print_timings:      sample time =       3.57 ms /     6 runs   (    0.59 ms per token,  1682.56 tokens per second)
llama_print_timings: prompt eval time =    3714.17 ms /   690 tokens (    5.38 ms per token,   185.78 tokens per second)
llama_print_timings:        eval time =     197.92 ms /     5 runs   (   39.58 ms per token,    25.26 tokens per second)
llama_print_timings:       total time =    4161.74 ms /   695 

Total score for approach 2 and distance function l2 is 0.4055727554179567
